<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/Window_Functions_MySQL_PostgreSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Install and SetUp

## MySQL

In [ ]:
!apt-get update > /dev/null
!apt-get -y install mysql-server > /dev/null
!/etc/init.d/mysql restart

 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


In [ ]:
!mysql --version

mysql  Ver 8.0.32-0buntu0.20.04.1 for Linux on x86_64 ((Ubuntu))


In [ ]:
!mysql -e 'DROP DATABASE IF EXISTS praxisdb;'
!mysql -e 'CREATE DATABASE praxisdb;'

In [ ]:
!mysql praxisdb -e 'show tables;'

In [ ]:
#!mysql praxisdb -e 'desc product_groups'

##PostgreSQL

In [1]:
# Install postgresql server
!apt update > /dev/null
!apt install postgresql > /dev/null

!service postgresql start





 * Starting PostgreSQL 12 database server
   ...done.


In [2]:
!sudo -u postgres psql -V

psql (PostgreSQL) 12.13 (Ubuntu 12.13-0ubuntu0.20.04.1)


In [3]:
# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
#
# Setup a database with name `sampledb` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS praxisdb;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE praxisdb;'

ALTER ROLE
NOTICE:  database "praxisdb" does not exist, skipping
DROP DATABASE
CREATE DATABASE


In [4]:
# Create file to store the password. Else will be prompted each time
!echo "localhost:5432:praxisdb:postgres:pass" > ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass

localhost:5432:praxisdb:postgres:pass


## Python Connection

In [5]:
import pandas as pd

!apt install libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
import MySQLdb

#!pip install psycopg2
import psycopg2


In [7]:
#conn = psycopg2.connect(database="praxisdb", user='postgres', password='pass', host='127.0.0.1', port= '5432')

In [8]:
def runSelect(s):
    #Set up connection
    #conn = MySQLdb.connect(db='praxisdb')
    conn = psycopg2.connect(database="praxisdb", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    resultdf = pd.read_sql(s, con=conn)
    conn.close()
    return resultdf

def runCmd(s):
    #Create Connection
    #conn = MySQLdb.connect(db='praxisdb')
    conn = psycopg2.connect(database="praxisdb", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

In [31]:
!ps -aef

UID          PID    PPID  C STIME TTY          TIME CMD
root           1       0  0 05:42 ?        00:00:00 /sbin/docker-init -- /datala
root           7       1  0 05:42 ?        00:00:03 /tools/node/bin/node /datala
root          18       0  0 05:42 ?        00:00:00 tail -n +0 -F /root/.config/
root          31       7  0 05:43 ?        00:00:00 /bin/bash -e /usr/local/cola
root          33       1  0 05:43 ?        00:00:00 /bin/bash -e /datalab/run.sh
root          35      33  0 05:43 ?        00:00:00 /usr/colab/bin/kernel_manage
root          63       7  0 05:43 ?        00:00:05 [python3] <defunct>
root          64       7  0 05:43 ?        00:00:00 python3 /usr/local/bin/colab
root          86       7  0 05:43 ?        00:00:02 /usr/bin/python3 /usr/local/
root          87       7  0 05:43 ?        00:00:00 /usr/local/bin/dap_multiplex
root         242      86  1 05:43 ?        00:00:07 /usr/bin/python3 -m ipykerne
root         277       1  0 05:43 ?        00:00:01 /usr/bin/p

# Create, Load Tables

In [9]:
ddl = 'drop table if exists product_groups;'
runCmd(ddl)

Command executed


In [10]:
ddl = 'drop table if exists products;'
runCmd(ddl)

Command executed


In [11]:
ddl = "\
CREATE TABLE product_groups (\
	group_id char(3) PRIMARY KEY,\
	group_name VARCHAR (255) NOT NULL\
);"
runCmd(ddl)


Command executed


In [12]:
ddl = "\
CREATE TABLE products (\
	product_id char(3) PRIMARY KEY,\
	product_name VARCHAR (255) NOT NULL,\
	price DECIMAL (11, 2),\
	group_id char(3) NOT NULL,\
	FOREIGN KEY (group_id) REFERENCES product_groups (group_id)\
);"
runCmd(ddl)

Command executed


In [13]:
ddl = "\
INSERT INTO product_groups (group_id,group_name) \
VALUES \
	('SP1','Smartphone'), \
	('LT2','Laptop'), \
	('TB3','Tablet'); \
"
runCmd(ddl)

Command executed


In [14]:
ddl = "\
INSERT INTO products (product_id,product_name, group_id,price) \
VALUES \
	('P11','Microsoft Lumia', 'SP1', 200),\
	('P12','HTC One', 'SP1', 400),\
	('P13','Nexus', 'SP1', 500),\
	('P14','iPhone', 'SP1', 900),\
	('P21','HP Elite', 'LT2', 1200),\
	('P22','Lenovo Thinkpad', 'LT2', 700),\
	('P23','Sony VAIO', 'LT2', 700),\
	('P24','Dell Vostro', 'LT2', 800),\
	('P31','iPad', 'TB3', 700),\
	('P32','Kindle Fire', 'TB3', 150),\
	('P33','Samsung Galaxy Tab', 'TB3', 200);"

runCmd(ddl)

Command executed


In [15]:
runSelect("select * from products")

,product_id,product_name,price,group_id
0,P11,Microsoft Lumia,200.0,SP1
1,P12,HTC One,400.0,SP1
2,P13,Nexus,500.0,SP1
3,P14,iPhone,900.0,SP1
4,P21,HP Elite,1200.0,LT2
5,P22,Lenovo Thinkpad,700.0,LT2
6,P23,Sony VAIO,700.0,LT2
7,P24,Dell Vostro,800.0,LT2
8,P31,iPad,700.0,TB3
9,P32,Kindle Fire,150.0,TB3


In [16]:
runSelect("select * from product_groups")

,group_id,group_name
0,SP1,Smartphone
1,LT2,Laptop
2,TB3,Tablet


#Windows Functions 
explanation for the SQL used here is available at this URL 
https://www.postgresqltutorial.com/postgresql-window-function/

In [17]:
runSelect("select * from products")

,product_id,product_name,price,group_id
0,P11,Microsoft Lumia,200.0,SP1
1,P12,HTC One,400.0,SP1
2,P13,Nexus,500.0,SP1
3,P14,iPhone,900.0,SP1
4,P21,HP Elite,1200.0,LT2
5,P22,Lenovo Thinkpad,700.0,LT2
6,P23,Sony VAIO,700.0,LT2
7,P24,Dell Vostro,800.0,LT2
8,P31,iPad,700.0,TB3
9,P32,Kindle Fire,150.0,TB3


In [26]:
sql = "\
SELECT \
	round(AVG (price),2) avgPrice \
FROM \
	products; \
"
runSelect(sql)

,avgprice
0,586.36


In [27]:
sql = "\
SELECT \
	group_name, \
	round(AVG (price),2) avgPrice \
FROM\
	products \
INNER JOIN product_groups USING (group_id) \
GROUP BY \
	group_name;\
"
runSelect(sql)

,group_name,avgprice
0,Smartphone,500.0
1,Tablet,350.0
2,Laptop,850.0


In [28]:
sql = "\
SELECT\
	product_name,\
	price,\
	group_name,\
	AVG (price)  OVER (\
	   PARTITION BY group_name \
	) AVGprice \
FROM\
	products\
	INNER JOIN \
		product_groups USING (group_id);\
"
runSelect(sql)

,product_name,price,group_name,avgprice
0,HP Elite,1200.0,Laptop,850.0
1,Lenovo Thinkpad,700.0,Laptop,850.0
2,Sony VAIO,700.0,Laptop,850.0
3,Dell Vostro,800.0,Laptop,850.0
4,Microsoft Lumia,200.0,Smartphone,500.0
5,HTC One,400.0,Smartphone,500.0
6,Nexus,500.0,Smartphone,500.0
7,iPhone,900.0,Smartphone,500.0
8,iPad,700.0,Tablet,350.0
9,Kindle Fire,150.0,Tablet,350.0


In [29]:
sql = "\
SELECT \
	product_name,\
	group_name,\
	price,\
	ROW_NUMBER () OVER (\
		PARTITION BY group_name\
		ORDER BY\
			price\
	) RowNumber \
FROM\
	products \
INNER JOIN product_groups USING (group_id);\
"
runSelect(sql)

,product_name,group_name,price,rownumber
0,Sony VAIO,Laptop,700.0,1
1,Lenovo Thinkpad,Laptop,700.0,2
2,Dell Vostro,Laptop,800.0,3
3,HP Elite,Laptop,1200.0,4
4,Microsoft Lumia,Smartphone,200.0,1
5,HTC One,Smartphone,400.0,2
6,Nexus,Smartphone,500.0,3
7,iPhone,Smartphone,900.0,4
8,Kindle Fire,Tablet,150.0,1
9,Samsung Galaxy Tab,Tablet,200.0,2


In [30]:
sql = "\
SELECT \
	product_name, \
	group_name, \
  price, \
	RANK () OVER ( \
		PARTITION BY group_name \
		ORDER BY \
			price \
	) prodRank \
FROM \
	products \
INNER JOIN product_groups USING (group_id); \
"
runSelect(sql)

,product_name,group_name,price,prodrank
0,Sony VAIO,Laptop,700.0,1
1,Lenovo Thinkpad,Laptop,700.0,1
2,Dell Vostro,Laptop,800.0,3
3,HP Elite,Laptop,1200.0,4
4,Microsoft Lumia,Smartphone,200.0,1
5,HTC One,Smartphone,400.0,2
6,Nexus,Smartphone,500.0,3
7,iPhone,Smartphone,900.0,4
8,Kindle Fire,Tablet,150.0,1
9,Samsung Galaxy Tab,Tablet,200.0,2


In [23]:
sql = "\
SELECT \
	product_name, \
	group_name, \
	price, \
	FIRST_VALUE (price) OVER ( \
		PARTITION BY group_name \
		ORDER BY \
			price \
	) AS lowest_price_per_group \
    , \
    LAST_VALUE (price) OVER ( \
		PARTITION BY group_name \
		ORDER BY \
			price RANGE BETWEEN UNBOUNDED PRECEDING \
		AND UNBOUNDED FOLLOWING \
	) AS highest_price_per_group \
FROM \
	products \
INNER JOIN product_groups USING (group_id); \
"

runSelect(sql)


,product_name,group_name,price,lowest_price_per_group,highest_price_per_group
0,Sony VAIO,Laptop,700.0,700.0,1200.0
1,Lenovo Thinkpad,Laptop,700.0,700.0,1200.0
2,Dell Vostro,Laptop,800.0,700.0,1200.0
3,HP Elite,Laptop,1200.0,700.0,1200.0
4,Microsoft Lumia,Smartphone,200.0,200.0,900.0
5,HTC One,Smartphone,400.0,200.0,900.0
6,Nexus,Smartphone,500.0,200.0,900.0
7,iPhone,Smartphone,900.0,200.0,900.0
8,Kindle Fire,Tablet,150.0,150.0,700.0
9,Samsung Galaxy Tab,Tablet,200.0,150.0,700.0


In [24]:
sql = "\
SELECT \
	product_name, \
	group_name, \
	price, \
	LAG (price, 1) OVER ( \
		PARTITION BY group_name \
		ORDER BY \
			price \
	) AS prev_price, \
	price - LAG (price, 1) OVER ( \
		PARTITION BY group_name \
		ORDER BY \
			price \
	) AS cur_prev_diff \
FROM \
	products \
INNER JOIN product_groups USING (group_id); \
"
runSelect(sql)

,product_name,group_name,price,prev_price,cur_prev_diff
0,Sony VAIO,Laptop,700.0,NaN,NaN
1,Lenovo Thinkpad,Laptop,700.0,700.0,0.0
2,Dell Vostro,Laptop,800.0,700.0,100.0
3,HP Elite,Laptop,1200.0,800.0,400.0
4,Microsoft Lumia,Smartphone,200.0,NaN,NaN
5,HTC One,Smartphone,400.0,200.0,200.0
6,Nexus,Smartphone,500.0,400.0,100.0
7,iPhone,Smartphone,900.0,500.0,400.0
8,Kindle Fire,Tablet,150.0,NaN,NaN
9,Samsung Galaxy Tab,Tablet,200.0,150.0,50.0


In [25]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Kolkata')))

Last tested   2023-01-27 11:20:05.400158+05:30


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)